This is a project to attempt to use machine learning to solve the integral 
$\frac{1}{x^2-x+1}dx$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


In [101]:
class ML(object):
    """
    class for implementing the machine learning process
    This will include the possibility of both a two layer model
    as well as an L-layer model.
    """
    
    ## constructor
    def __init__(self, size, input_layer, true_values, testing_input, testing_true, learning_rate = 0.0075, hidden_function = 'relu', last_function = 'sigmoid', num_iterations = 3000, print_cost = False, plot_cost = False):
        self.size = size
        self.X = input_layer
        self.Y = true_values
        self.test_X = testing_input
        self.test_Y = testing_true
        self.learning_rate = learning_rate
        self.last_function = last_function
        self.hidden_function = hidden_function
        self.num_iterations = num_iterations
        self.print_cost = False
        self.dimensions = []
        self.final_paramters = {}
        self.plot_cost_bool = plot_cost

    ## method for recieving number of dimensions for each layer
    ## TODO add a posibility to iterate and find ideal dimensions
    def set_layers_dims(self, layers_dims):
        if len(layers_dims) != self.size:
            print("you must enter dimensions for "+str(size)+" layers or reset the size")
        else:
            self.dimensions = layers_dims
            
    ##methods to update the various varaibles
    def set_learning_rate(self, rate):
        self.learning_rate = rate
        
    def set_last_function(self, function):
        self.last_function = function
        
    def set_hidden_function(self, function):
        self.hidden_function = function
        
    def set_num_iterations(self, num):
        self.num_iterations = num
        
    def set_print_cost(self, boo):
        self.print_cost = boo
        
    def set_plot_cost(self, boo):
        self.plot_cost = boo
        
    def set_final_params(self, params):
        self.final_parameters = params
    
    ## set hidden layer activation function if different then default
    ## options limited to the functions coded inside
    def hidden_layer_function(self, function):
        if ((function != 'relu') and  (function != 'sigmoid') and  (function != 'softmax')):
            print("function must be 'relu', 'sigmoid', or 'softmax', new functions may be added in the future")
        else:
            self.hidden_function = function

    ## set output layer activation function if different then default
    ## options limited to the functions coded inside
    def last_layer_function(self, function):
        if ((function != 'relu') and  (function != 'sigmoid') and  (function != 'softmax')):
            print("function must be 'relu', 'sigmoid', or 'softmax', new functions may be added in the future")
        else:
            self.last_function = function
    
    ## definitions for the activation layers functions
    ## also definitions for the backward propogation using derivitives
    ## currently includes relu and sigmoid
    ## TODO soft_max
    def __sigmoid(self, Z):
        Z = Z.astype(float)
        A = 1/(1+np.exp(-Z))
        activation_cache = Z
        return A, activation_cache
    
    def __relu(self, Z):
        A = np.maximum(0,Z)
        activation_cache = Z
        return A, activation_cache
    
    def __soft_max(self, Z):
        num_rows, num_columns = Z.shape
        activation_cache = Z
        A = Z*0
        exps = np.exp(Z)
        exps_sums = np.zeros((1,Z.shape[1]))
        exps_sums = np.sum(exps, axis = 0)
        for i in range(num_columns):
            for j in range(num_rows):
                A[j][i] = exps[j][i]/exps_sums[i]
        return A, activation_cache
    
    def __sigmoid_backward(self, dA, activation_cache):
        S, Z = self.__sigmoid(activation_cache)
        dZ = dA * (S*(1.0-S))
        return dZ
    
    def __relu_backward(self, dA, activation_cache):
        Z = activation_cache
        num_rows, num_columns = Z.shape
        dZ = Z * 0
        for i in range(num_rows):
            for j in range(num_columns):
                if Z[i][j] <= 0:
                    dZ[i][j] = 0
                elif Z[i][j] > 0:
                    dZ[i][j] = dA[i][j]
        return dZ
    
    def __soft_max_backward(self, dA, activation_cache):
        S, Z = self.__soft_max(activation_cache)
        m, n = Z.shape
        p = soft_max(Z)
        # outer products
        # (p1^2  p1*p2 p1*p3 ...)
        # (p2*p1 p2^2  p2*p3 ...)
        # (...                  )
        tensor1 = np.einsum('ij,ik->ijk',p,p)
        # (n,n) identitity of feature vector
        # (p1 0  0 ...)
        # (0  p2 0 ...)
        # (...        )
        tensor2 = np.einsum('ij,jk->ijk',p,np.eye(n,n))
    
        dSoftmax = tensor2 - tensor1
        dZ = np.einsum('ijk,ik->ij', dSoftmax, dA)
        return dZ

    ## initialises weights and bias variables for a two-layer NN
    def initialize_parameters(self, n_x, n_h, n_y):
        np.random.seed(1)
        
        W1 = np.random.randn(n_h, n_x) * 0.01
        b1 = np.zeros((n_h, 1))
        W2 = np.random.randn(n_y, n_h) * 0.01
        b2 = np.zeros((n_y,1))
    
        parameters = {"W1": W1,
                      "b1": b1,
                      "W2": W2,
                      "b2": b2}
        return parameters
    
    ## linear forward propogation step, taken at each layer
    def linear_forward(self, A, W, b):
        Z = np.dot(W,A) + b
        cache = (A,W,b)
        return Z, cache
    
    ## forward propagation for activation step taken at each layer
    def linear_activation_forward(self, A_prev, W, b, activation):
        if activation == "sigmoid":
            Z, linear_cache = self.linear_forward(A_prev, W, b)
            A, activation_cache = self.__sigmoid(Z)
        
        elif activation == "relu":
            Z, linear_cache = self.linear_forward(A_prev, W, b)
            A, activation_cache = self.__relu(Z)
        
        elif activation == "soft_max":
            Z, linear_cache = self.linear_forward(A_prev, W, b)
            A, activation_cache = self.__soft_max(Z)
        
        cache = (linear_cache, activation_cache)
    
        return A, cache
    
    ## cost function
    def __compute_cost(self, AL, Y):
        m = Y.shape[1]*Y.shape[0]
        cost = -(np.sum(Y*np.log(AL) + (1-Y)*np.log(1-AL)))/m
        cost = np.squeeze(cost)
        return cost
    
    ## linear backward propogation step, taken at each layer
    def linear_backward(self, dZ, cache):
        A_prev, W, b = cache
        m = A_prev.shape[1]
    
        dW = (np.dot(dZ,A_prev.T))/m
        db = np.sum(dZ, axis=1, keepdims=True)/m
        dA_prev = np.dot(W.T, dZ)
    
        return dA_prev, dW, db
    
    ## backward propagation for activation step taken at each layer
    def linear_activation_backward(self, dA, cache, activation):
        linear_cache, activation_cache = cache
    
        if activation == "relu":
            dZ = self.__relu_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        
        elif activation == "sigmoid":
            dZ = self.__sigmoid_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        
        elif activation == "soft_max":
            dZ = self.__soft_max_backward(dA, activation_cache)
            dA_prev, dW, db = self.linear_backward(dZ, linear_cache)
        
        return dA_prev, dW, db
    
    ## update parameters using gradient descent
    def update_parameters(self, params, grads, learning_rate):
        parameters = params.copy()
        L = len(parameters) // 2 #num layers
    
        for l in range(L):
            parameters["W" + str(l+1)] = parameters["W"+str(l+1)] - learning_rate*grads["dW" + str(l+1)]
            parameters["b" + str(l+1)] = parameters["b"+str(l+1)] - learning_rate*grads["db" + str(l+1)]
        
        return parameters
    
    ## runs the two layer NN 
    def two_layer_model(self, X, Y):
        np.random.seed(1)
        grads = {}
        costs = []
        m = X.shape[1]
        (n_x, n_h, n_y) = self.dimensions
        
        parameters = self.initialize_parameters(n_x, n_h, n_y)
    
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        
        for i in range(0, self.num_iterations):
            A1, cache1 = self.linear_activation_forward(X, W1, b1, self.hidden_function)
            A2, cache2 = self.linear_activation_forward(A1, W2, b2, self.last_function)
            cost = self.__compute_cost(A2, Y)
            dA2 = -(np.divide(Y, A2) - np.divide(1-Y, 1-A2))
            dA1, dW2, db2 = self.linear_activation_backward(dA2, cache2, self.last_function)
            dA0, dW1, db1 = self.linear_activation_backward(dA1, cache1, self.hidden_function)
            grads['dW1'] = dW1
            grads['db1'] = db1
            grads['dW2'] = dW2
            grads['db2'] = db2
            
            parameters = self.update_parameters(parameters, grads, self.learning_rate)
            
            W1 = parameters["W1"]
            b1 = parameters["b1"]
            W2 = parameters["W2"]
            b2 = parameters["b2"]
        
            if self.print_cost and (i % 100 == 0 or i == self.num_iterations - 1):
                print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
            if i % 100 == 0 or i == self.num_iterations:
                costs.append(cost)
            
        return parameters, costs
    
    def plot_costs(self, costs):
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per hundreds)')
        plt.title("Learning rate =" + str(self.learning_rate))
        plt.show()
        
    def predict(self, parameters, X, Y, option="binary"):
        W1 = parameters["W1"]
        b1 = parameters["b1"]
        W2 = parameters["W2"]
        b2 = parameters["b2"]
        A1, cache1 = self.linear_activation_forward(X, W1, b1, self.hidden_function)
        A2, cache2 = self.linear_activation_forward(A1, W2, b2, self.last_function)
        correct = 0
        total = 0
        if option == "binary":
            A2 = np.round(A2)
            num_rows, num_columns = A2.shape
            for i in range(num_rows):
                for j in range(num_columns):
                    if A2[i][j] == Y[i][j]:
                        correct += 1
                    total +=1
                
        if option == "multiclass":
            Y_arg = np.argmax(Y, axis = 0)
            A2_arg = np.argmax(A2, axis = 0)
            for i in range(len(Y_arg)):
                if Y_arg[i] == A2_arg[i]:
                    correct += 1
                total += 1
        print("total = "+str(total)+" correct = "+str(correct))
        print("Accuracy = "+str(correct/total))
        return correct/total
    
    def run_machine(self):
        self.final_parameters, costs = self.two_layer_model(self.X, self.Y)
        if self.plot_cost_bool:
            self.plot_costs(costs)
        
    def machine_accuracy(self):
        self.predict(self.final_parameters, self.X, self.Y, option="multiclass")
        self.predict(self.final_parameters, self.test_X, self.test_Y, option="multiclass")
    
            
            